In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
import time
import tracemalloc

In [2]:
class BaseClassifier:
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)
        self.train_data, self.test_data = self.split_data()
        self.train_features, self.train_labels, self.test_features, self.test_labels = self.prepare_features()

    def split_data(self, test_size=0.3):
        return train_test_split(self.data, test_size=test_size)

    def prepare_features(self):
        train_features = self.train_data[[
            'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
        train_labels = self.train_data.Species
        test_features = self.test_data[[
            'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
        test_labels = self.test_data.Species
        return train_features, train_labels, test_features, test_labels

    def evaluate(self, predictions, true_labels):
        accuracy = metrics.accuracy_score(predictions, true_labels)
        print(
            f'The accuracy of the {self.__class__.__name__} is: {accuracy * 100:.2f}%')
        return accuracy

    def time_method(self, method, *args, **kwargs):
        """Helper method to time any class method and return (result, duration_ms)"""
        start_time = time.time()
        result = method(*args, **kwargs)
        end_time = time.time()
        duration_ms = (end_time - start_time) * 1000  # Convert to milliseconds
        return result, duration_ms

In [3]:
class SVMClassifier(BaseClassifier):
    def __init__(self, data_path):
        super().__init__(data_path)
        # Composition: Compose an SVM model inside the class
        self.model = svm.SVC()  

    def train(self):
        tracemalloc.start()
        start_time = time.time()
        
        self.model.fit(self.train_features, self.train_labels)
        
        self.training_time = time.time() - start_time
        self.memory_usage = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()

    def predict(self):
        start_time = time.time()
        predictions = self.model.predict(self.test_features)
        self.prediction_time = time.time() - start_time
        return predictions

In [4]:
class LogisticRegressionClassifier(BaseClassifier):
    def __init__(self, data_path):
        super().__init__(data_path)
        # Composition: Compose a Logistic Regression model inside the class
        self.model = LogisticRegression()

    def train(self):
        tracemalloc.start()
        start_time = time.time()

        self.model.fit(self.train_features, self.train_labels)

        self.training_time = time.time() - start_time
        self.memory_usage = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()

    def predict(self):
        start_time = time.time()
        predictions = self.model.predict(self.test_features)
        self.prediction_time = time.time() - start_time
        return predictions

In [5]:
class DecisionTreeClassifierStyle(BaseClassifier):
    def __init__(self, data_path):
        super().__init__(data_path)
        # Composition: Compose a Decision Tree model inside the class
        self.model = DecisionTreeClassifier()

    def train(self):
        tracemalloc.start()
        start_time = time.time()

        self.model.fit(self.train_features, self.train_labels)

        self.training_time = time.time() - start_time
        self.memory_usage = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()

    def predict(self):
        start_time = time.time()
        predictions = self.model.predict(self.test_features)
        self.prediction_time = time.time() - start_time
        return predictions

In [6]:
class KNNClassifier(BaseClassifier):
    def __init__(self, data_path, n_neighbors=3):
        super().__init__(data_path)
        # Composition: Compose a KNN model inside the class
        self.model = KNeighborsClassifier(n_neighbors=n_neighbors)

    def train(self):
        tracemalloc.start()
        start_time = time.time()

        self.model.fit(self.train_features, self.train_labels)

        self.training_time = time.time() - start_time
        self.memory_usage = tracemalloc.get_traced_memory()[1]
        tracemalloc.stop()

    def predict(self):
        start_time = time.time()
        predictions = self.model.predict(self.test_features)
        self.prediction_time = time.time() - start_time
        return predictions

In [7]:
def run_classification():
    classifiers = [
        SVMClassifier("./Input_Data/Iris.csv"),
        LogisticRegressionClassifier("./Input_Data/Iris.csv"),
        DecisionTreeClassifierStyle("./Input_Data/Iris.csv"),
        KNNClassifier("./Input_Data/Iris.csv", n_neighbors=3)
    ]

    results = []

    for classifier in classifiers:
        print(f"\n{classifier.__class__.__name__}")

        # Time the training
        _, train_time = classifier.time_method(classifier.train)
        print(f"Training time: {train_time:.2f} ms")
        print(
            f"Memory usage during training: {classifier.memory_usage / 1024:.2f} KB")

        # Time the prediction
        predictions, predict_time = classifier.time_method(classifier.predict)
        print(f"Prediction time: {predict_time:.2f} ms")

        # Evaluate accuracy
        accuracy = classifier.evaluate(predictions, classifier.test_labels)

        # Store results
        results.append({
            'Classifier': classifier.__class__.__name__,
            'Accuracy': accuracy,
            'TrainTime_ms': train_time,
            'PredictTime_ms': predict_time,
            'TotalTime_ms': train_time + predict_time, 
            'MemoryUsage_KB': classifier.memory_usage / 1024
        })

    # Print summary table
    # print("\nSummary:")
    # print(pd.DataFrame(results).to_string(index=False))

In [8]:
run_classification()


SVMClassifier
Training time: 12.89 ms
Memory usage during training: 24.54 KB
Prediction time: 2.82 ms
The accuracy of the SVMClassifier is: 91.11%

LogisticRegressionClassifier
Training time: 490.58 ms
Memory usage during training: 250.40 KB
Prediction time: 6.13 ms
The accuracy of the LogisticRegressionClassifier is: 95.56%

DecisionTreeClassifierStyle
Training time: 12.48 ms
Memory usage during training: 19.80 KB
Prediction time: 1.58 ms
The accuracy of the DecisionTreeClassifierStyle is: 95.56%

KNNClassifier
Training time: 6.44 ms
Memory usage during training: 21.30 KB
Prediction time: 8.28 ms
The accuracy of the KNNClassifier is: 97.78%


/home/shuchi/College/OOP_Code/Research_Project_Code/myenv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
